# Predicting Corporación Favorita sales through varieties of condition


Goals:


Forecast store sales by building model to predicts unit sales for each items sold at different stores



# 1. Import packages

In [2]:
#importing packages
#basics
import numpy as np
import pandas as pd
import os
import gc
import warnings

#visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


# 2. Read the data

In [17]:
train = pd.read_csv(r"C:\Users\rzzkt\OneDrive - Institut Teknologi Bandung\Documents\Rezki\projects\store-sales-time-series-forecasting\train.csv")
test = pd.read_csv(r"C:\Users\rzzkt\OneDrive - Institut Teknologi Bandung\Documents\Rezki\projects\store-sales-time-series-forecasting\test.csv")
stores = pd.read_csv(r"C:\Users\rzzkt\OneDrive - Institut Teknologi Bandung\Documents\Rezki\projects\store-sales-time-series-forecasting\stores.csv")
transactions = pd.read_csv(r"C:\Users\rzzkt\OneDrive - Institut Teknologi Bandung\Documents\Rezki\projects\store-sales-time-series-forecasting\transactions.csv")
oil = pd.read_csv(r"C:\Users\rzzkt\OneDrive - Institut Teknologi Bandung\Documents\Rezki\projects\store-sales-time-series-forecasting\oil.csv")
holiday = pd.read_csv(r"C:\Users\rzzkt\OneDrive - Institut Teknologi Bandung\Documents\Rezki\projects\store-sales-time-series-forecasting\holidays_events.csv")

In [4]:
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [5]:
test.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [6]:
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [7]:
transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [8]:
oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [18]:
holiday.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


# 3. Data cleaning
This step include, check for null value in the dataframe, unnecessary duplicated values, and drop columns that redundant. 

In [15]:
# check is there a null value in dataframe
# train.isna().sum()
# test.isna().sum()
oil.isna().sum()

date           0
dcoilwtico    43
dtype: int64

In [13]:
# change the datetime type for train dataframe
train['day']=pd.to_datetime(train['date']).dt.day
train['month']=pd.to_datetime(train['date']).dt.month
train['year']=pd.to_datetime(train['date']).dt.year

# change the datetime type for test dataframe
test['day']=pd.to_datetime(test['date']).dt.day
test['month']=pd.to_datetime(test['date']).dt.month
test['year']=pd.to_datetime(test['date']).dt.year

# change the datetime type for 

,id,date,store_nbr,family,sales,onpromotion,month,day,year
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,1,1,2013
1,1,2013-01-01,1,BABY CARE,0.000,0,1,1,2013
2,2,2013-01-01,1,BEAUTY,0.000,0,1,1,2013
3,3,2013-01-01,1,BEVERAGES,0.000,0,1,1,2013
4,4,2013-01-01,1,BOOKS,0.000,0,1,1,2013
...,...,...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,8,15,2017
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,8,15,2017
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,8,15,2017
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,8,15,2017
